In [2]:
import whisper

In [3]:
whisper_tiny = whisper.load_model("tiny.en")

c:\Users\keith\Desktop\repos\whisper-streaming\private-whisper\whisper\__init__.py:151: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_locatio

In [4]:
print(whisper_tiny.dims)

ModelDimensions(n_mels=80, n_audio_ctx=1500, n_audio_state=384, n_audio_head=6, n_audio_layer=4, n_vocab=51864, n_text_ctx=448, n_text_state=384, n_text_head=6, n_text_layer=4)


In [5]:
from whisper.streaming import StreamingConfig

model_dims = whisper_tiny.dims
streaming_config = StreamingConfig()
model = whisper.WhisperStreaming(model_dims, streaming_config)

In [6]:
model.load_state_dict(whisper_tiny.state_dict(), strict=False)

_IncompatibleKeys(missing_keys=['decoder.blocks.0.p_choose_layer.energy_bias', 'decoder.blocks.0.p_choose_layer.q_energy_proj.layers.0.weight', 'decoder.blocks.0.p_choose_layer.q_energy_proj.layers.0.bias', 'decoder.blocks.0.p_choose_layer.q_energy_proj.layers.2.weight', 'decoder.blocks.0.p_choose_layer.q_energy_proj.layers.2.bias', 'decoder.blocks.0.p_choose_layer.q_energy_proj.layers.4.weight', 'decoder.blocks.0.p_choose_layer.q_energy_proj.layers.4.bias', 'decoder.blocks.0.p_choose_layer.q_energy_proj.layers.6.weight', 'decoder.blocks.0.p_choose_layer.q_energy_proj.layers.6.bias', 'decoder.blocks.0.p_choose_layer.k_energy_proj.layers.0.weight', 'decoder.blocks.0.p_choose_layer.k_energy_proj.layers.0.bias', 'decoder.blocks.0.p_choose_layer.k_energy_proj.layers.2.weight', 'decoder.blocks.0.p_choose_layer.k_energy_proj.layers.2.bias', 'decoder.blocks.0.p_choose_layer.k_energy_proj.layers.4.weight', 'decoder.blocks.0.p_choose_layer.k_energy_proj.layers.4.bias', 'decoder.blocks.0.p_choos

In [7]:
print(model.decoder.blocks[0].p_choose_layer.q_energy_proj.layers[0])

Linear(in_features=384, out_features=384, bias=True)


In [11]:
print(model.transcribe_stream("./jfk.wav", chunk_size=32000))

stream source yield
stream source yield
log mel yield
stream source yield
log mel yield
encoder len torch.Size([80, 200])
stream source yield
log mel yield
encoder len torch.Size([80, 400])
READ
Remove Hooks
<|startoftranscript|>
Install hooks
break read 0.07628928124904633 False
stream source yield
log mel yield
encoder len torch.Size([80, 600])
READ
Remove Hooks
<|startoftranscript|>
Install hooks
break read 0.07631254196166992 False
stream source yield
log mel yield
encoder len torch.Size([80, 800])
READ
Remove Hooks
<|startoftranscript|>
Install hooks
break read 0.07631116360425949 False
log mel yield
encoder len torch.Size([80, 1000])
READ
Remove Hooks
<|startoftranscript|>
Install hooks
break read 0.07633952051401138 False
encoder len torch.Size([80, 1100])
READ
Remove Hooks
<|startoftranscript|>
Install hooks
break read 0.07631587237119675 False
READ
Remove Hooks
<|startoftranscript|>
Install hooks
<|startoftranscript|>
<|startoftranscript|> And
<|startoftranscript|> And so
<|st

In [ ]:
audio = whisper.load_audio(file="jfk.wav", sr=16000)
audio = whisper.pad_or_trim(audio)

mel = whisper.log_mel_spectrogram(audio, n_mels=model.dims.n_mels)

mel = mel.unsqueeze(0)
xa = model.encoder(mel)

print(mel.shape)
print(xa.shape) 

torch.Size([1, 80, 3000])
torch.Size([1, 1500, 384])


In [97]:
import torch

def monotonic_alignment(p):  # p: [B, T, S]
    bsz, tgt_len, src_len = p.size()
    
    # Extension probability matrix: roll, unsqueeze, expand, upper-triangular
    p_ext = p.roll(1, [-1]).unsqueeze(-2).expand(-1, -1, src_len, -1).triu(1)
    
    # Transition matrix: 1 - p_ext, then cumulative product, upper-triangular
    T = (1 - p_ext).cumprod(-1).triu()
    
    # Base case for alpha: alpha_0 = p_0 * T_0
    alpha = [p[:, [0]] * T[:, [0]]]
    
    # Recurrence over target steps
    for i in range(1, tgt_len):
        print(alpha[i - 1].size())
        print(T[:, i].size())
        alpha.append(p[:, [i]] * torch.bmm(alpha[i - 1], T[:, i]))

    return torch.cat(alpha[1:], dim=1)

In [99]:
import whisper
from whisper.tokenizer import get_tokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"

# load audio and encode
audio = whisper.load_audio("jfk.wav", sr=16000)
audio = whisper.pad_or_trim(audio)
mel = whisper.log_mel_spectrogram(audio, n_mels=model.dims.n_mels)

with torch.no_grad():
    xa = model.encoder(mel.unsqueeze(0))  # (1, n_audio_ctx, n_audio_state)

# initialize tokens
tokenizer = get_tokenizer(multilingual=False)
tokens = torch.tensor([tokenizer.sot_sequence], device=device)  # (1, T)

for time_step in range(100):  # max steps
    logits, p_choose = model.decoder.decode_with_pchoose(tokens, xa)
    
    _, tgt_len, src_len = p_choose.size()

    p_choose = p_choose.view(model.dims.n_text_layer, -1, tgt_len, src_len)
    print(p_choose.size())
    
    if time_step > 1:
        alpha = monotonic_alignment(p_choose)
        print(alpha)

    next_token = logits[:, -1].argmax(dim=-1, keepdim=True)
    # print(tokenizer.decode(next_token[0].tolist()))

    tokens = torch.cat([tokens, next_token], dim=-1)
    
    if time_step == 3:
        break

    if next_token.item() == tokenizer.eot:
        break

# decode final output
print(len(model.decoder.blocks))
print("Decoded text:", tokenizer.decode(tokens[0].tolist()))


torch.Size([4, 6, 1, 750])
torch.Size([4, 6, 2, 750])
torch.Size([4, 6, 3, 750])


ValueError: too many values to unpack (expected 3)

In [60]:
tokens = torch.tensor([tokenizer.sot_sequence]) 

for _ in range(100):
    logits, _ = model.decoder.decode_with_pchoose(tokens, xa)
    next_token = logits[:, -1].argmax(dim=-1, keepdim=True)  # greedy decode
    tokens = torch.cat([tokens, next_token], dim=-1)
    print(tokens)

    if next_token.item() == tokenizer.eot:
        break

tensor([[50258, 50259, 50359, 50257]])


In [61]:
tokens = tokens.squeeze(0)
tokenizer.decode(tokens)

'<|startoftranscript|><|en|><|transcribe|><|endoftext|>'